# Trabalho Prático 3
**Grupo 22**

Alexis Correia - A102495 <br>
João Fonseca - A102512 <br>

## Problema 1

O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 

> Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.

```Python
{INPUT  a, b}
{assume  a > 0 and b > 0}
0: r, r', s, s', t, t' = a, b, 1, 0, 0, 1
1: while r' != 0
2:   q = r div r'
3:   r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
{OUTPUT r, s, t}
```

1. Construa um SFOTS usando BitVector’s de tamanho $n$ que descreva o comportamento deste programa.  Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.
2. Prove, usando a metodologia dos invariantes e interpolantes, que o modelo nunca atinge o estado de erro.

## Resolução

Um SFOTS é definido  por $\:\:\Sigma\;\equiv\;\langle\,\mathsf{X}\,,\,\mathsf{next}\,,\,\mathsf{I}\,,\,\mathsf{T}\,,\,\mathsf{E}\,\rangle\:\:$. Similiar a um FOTS, os estados são constituídos pelas variáveis do programa mais o __program counter__ (pc) e, tanto o estado inicial quanto as relações de trnsição, são caracterizados por predicados. A maior diferença se encontra na existência do estado de erro.

- As variáveis do programa são: $ r $, $ r' $, $ s $, $ s' $, $ t $, $ t' $, $ q $ e não podemos nos esquecer do $ a $ e do $ b $
- O estado inicial é caracterizado pelo predicado: $\mathit{pc} = 0 \wedge a \ge 0 \wedge b \ge 0$
- As transições possíveis são caracterizadas das seguintes formas:
$$
\begin{array}{c}
(\mathit{pc}=0\wedge a\ge 0\wedge b\ge 0\wedge\mathit{PC}=1\wedge A=a\wedge B=b\wedge R=a\wedge R'=b\wedge S=1\wedge S'=0\wedge T=0\wedge T'=1)
\\\vee\\
(\mathit{pc}=1\wedge r'=0\wedge\mathit{PC}=4\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t')
\\\vee\\
(\mathit{pc}=1\wedge r'\neq 1\wedge\mathit{PC}=2\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t')
\\\vee\\
(\mathit{pc}=2\wedge\mathit{PC}=3\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge q = r\;\;\text{div}\;\; r')
\\\vee\\
(\mathit{pc}=3\wedge\mathit{PC}=1\wedge A=a\wedge B=b\wedge R=r'\wedge R'=r-q\times r'\wedge S=s'\wedge S'=s-q\times s'\wedge T=t'\wedge T'=t-q\times t')
\end{array}
$$
- O estado de erro acontece caso haja um ``overflow`` em alguma das variáveis ou quando $ r=0 $

> Repare que: nas transições, dado que as variáveis $ r' $, $ s' $ e $ t' $ são variáveis do programa, representamos as variáveis do próximo estado por letras maiúsculas.

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import BVType, INT

n = 32 # num bits = 4 bytes

# SFOTS #

def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i), INT)
    state['a'] = Symbol('a'+str(i), BVType(n)) ###
    state['b'] = Symbol('b'+str(i), BVType(n)) ###
    state['r'] = Symbol('r'+str(i), BVType(n))
    state['r\''] = Symbol('r\''+str(i), BVType(n))
    state['s'] = Symbol('s'+str(i), BVType(n))
    state['s\''] = Symbol('s\''+str(i), BVType(n))
    state['t'] = Symbol('t'+str(i), BVType(n))
    state['t\''] = Symbol('t\''+str(i), BVType(n))
    state['q'] = Symbol('q'+str(i), BVType(n)) ###
    return state

def init(state):
    A = BVSGE(state['a'], BV(0,n)) 
    B = BVSGE(state['b'], BV(0,n))
    C = Equals(state['pc'], Int(0))
    return And(A,B,C)

def trans(curr, prox):
    t01 = And(Equals(curr['pc'],Int(0)), BVSGE(curr['a'],BV(0,n)), BVSGE(curr['b'],BV(0,n)), Equals(prox['pc'],Int(1)),
              Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']), Equals(prox['r'],curr['a']), Equals(prox['r\''],curr['b']),
              Equals(prox['s'],Int(1)), Equals(prox['s\''],Int(0)), Equals(prox['t'],Int(0)), Equals(prox['t'], Int(1)))
    t14 = And(Equals(curr['pc'],Int(0)), Equals(curr['r\''], BV(0,n)), Equals(prox['pc'],Int(4)), Equals(prox['a'],curr['a']),
              Equals(prox['b'],curr['b']), Equals(prox['r'],curr['r']), Equals(prox['r\''],curr['r\'']),Equals(prox['s'],curr['s']),
              Equals(prox['s\''],curr['s\'']), Equals(prox['t'],curr['t']), Equals(prox['t'], curr['t\'']))
    t12 = And()
    t23 = And()
    t31 = And()
    return Or(t01, t14, t12, t23, t31)

def error(state):
    #overflow
    R = Equals(state['r'], BV(0, n))
    return R # Or overflow